<a href="https://colab.research.google.com/github/ferustigris/colab_invest/blob/master/Update_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

Install dependencies for Yahoo finance and gsheets. As well as client_secret.json required in root directory. Could be downloaded from [here](https://console.cloud.google.com/apis/api/drive.googleapis.com/metrics?project=gold-access-268714)

In [1]:
!pip install pygsheets

In [2]:
!pip install yfinance

# Framework

Create Yahoo API reader:

In [3]:
import yfinance as yf

BLNS = 1000_000_000

class NoValue(Exception):
  pass

class YahooFinanceReader:
    """Yahoo Finance Reader class for access via API"""

    class Ticker:
        """Case-class for the particular ticket"""

        def __init__(self, tkt):
            """Pass ticket object in the constructor"""
            self.tkt = tkt
            print(tkt.info)

        def get_current_price(self):
            """Get very current price of the ticket"""
            return self.tkt.info["currentPrice"]

        def get_divs(self):
            """Get annual divs of the ticket"""
            div = None
            try:
              divs = self.tkt.info["fiveYearAvgDividendYield"]
              if not divs:
                try:
                  divs = self.tkt.info["trailingAnnualDividendYield"]
                except KeyError:
                  print("trailingAnnualDividendYield: key error")
              if not divs:
                try:
                  divs = self.tkt.info["dividendYield"]
                except KeyError:
                  print("dividendYield: key error")
              if not divs:
                try:
                  divs = self.tkt.info["forwardAnnualDividendYield"]
                except KeyError:
                  print("forwardAnnualDividendYield: key error")
              return divs
            except KeyError:
              print("dividendYield: key error")
              raise NoValue("ERROR: %s" % "netin key error")

        def get_net(self):
            """Get annual netIncomeToCommon of the ticket in billions"""
            print(self.tkt.info)
            try:
                return self.tkt.info["netIncomeToCommon"] / BLNS
            except KeyError:
                print("ERROR: %s" % "netin key error")
            raise NoValue("ERROR: %s" % "netin key error")

        def get_revenue(self):
            try:
                """Get annual totalRevenue of the ticket in billions"""
                return self.tkt.info["totalRevenue"] / BLNS
            except KeyError:
                print("ERROR: %s" % "Total revenue key error")
            raise NoValue("ERROR: %s" % "Total revenue key error")

        def get_total_assets(self):
            """Get total assets of the ticket in billions"""
            return self.tkt.info["totalAssets"] / BLNS

        def get_pe(self):
            """Get total assets of the ticket in billions"""
            return self.tkt.info["trailingPE"]

        def get_fpe(self):
            """Get total assets of the ticket in billions"""
            try:
              return self.tkt.info["forwardingPE"]
            except KeyError:
              return self.tkt.info["forwardPE"]

        def get_freecash(self):
            """Get freeCashflow of the ticket in billions"""
            return self.tkt.info["freeCashflow"] / BLNS

        def get_shares(self):
            """Get shares volume of the ticket in billions"""
            return self.tkt.info["impliedSharesOutstanding"] / BLNS

        def get_sma200days(self):
            """Get twoHundredDayAverage of the ticket in billions"""
            return self.tkt.info["twoHundredDayAverage"] / BLNS

        def get_history(self):
          return self.tkt.history(period="max")

        def get_debt(self):
            """Get annual total Liabilities of the ticket in billions"""
            try:
                return self.tkt.balance_sheet['Total Liabilities'] / BLNS
            except KeyError:
                print("ERROR: %s" % "Total Liabilities key error")
            raise NoValue("ERROR: %s" % "Total Liabilities key error")

        def get_assets(self):
           return self.tkt.balance_sheet['Net Tangible Assets'] / BLNS

    def get_ticker(self, ticker_name):
        """Returns ticket object by ticket name"""
        print("Getting ticket object for %s..." % ticker_name)
        return self.Ticker(yf.Ticker(ticker_name))


Create gsheet writer, the file [itself](https://docs.google.com/spreadsheets/d/1hc8kcZJDUJ91qL-mp_NDxZUkrOnhihdzNhxVKRF04fQ/edit#gid=1283370971):

In [4]:
import pygsheets

# authorization
MAX_AMOUNT_ROWS = 50
COL_AVG = "" #TODO twoHundredDayAverage
COL_PRICE = 'E'
COL_DIVS = 'V'
COL_REVENUE = 'L'
COL_NET = 'M'
COL_ASSETS = 'O'
COL_SHARES = 'D'
COL_AVG = 'F'
COL_PE = "W"
COL_FPE = "X"
COL_DEBT = "S"
COL_FREECASH = "Y"
NYSE_SHEET_NUMBER = 1
EU_SHEET_NUMBER = 2
PATH_TO_KEY_JSON = 'client_secret.json'
GSHEET_NAME = 'Finance stock and invest'


class GSheetUpdater:
    """Class for Finance stock and invest updates"""
    def __init__(self, sheet=NYSE_SHEET_NUMBER):
        self.__gc = pygsheets.authorize(service_file=PATH_TO_KEY_JSON)
        # open the google spreadsheet (by is the name of my sheet)
        self.__sh = self.__gc.open(GSHEET_NAME)

        # select the right sheet
        self.__wks = self.__sh[sheet]

    def apply_for_all(self, func, col_to_be_updated):
        """applies function for all rows"""
        for i in range(2, MAX_AMOUNT_ROWS):
            header = self.__wks.cell('B%i' % i)
            ticket = header.value
            try:
              self.__wks.update_value('%s%i' % (col_to_be_updated, i), try_func(ticket))
            except NoValue:
              self.__wks.update_value('%s%i' % (col_to_be_updated, i), None)
              print("ERROR: %s" % s)

    def apply_for_all(self, cols_and_funcs, start_from=2):
        """applies function for all rows"""
        for i in range(start_from, MAX_AMOUNT_ROWS):
            print("Index=%i" % i)
            header = self.__wks.cell('B%i' % i)
            ticket = header.value
            if not ticket:
              raise NoValue("No ticket found")
            for (col_to_be_updated, func) in cols_and_funcs.items():
              for tries in range(3):
                try:
                  self.__wks.update_value('%s%i' % (col_to_be_updated, i), func(ticket))
                  break
                except NoValue as e:
                    self.__wks.update_value('%s%i' % (col_to_be_updated, i), None)
                    print("ERROR NoValue: %s" % e)
                except KeyError as e:
                    self.__wks.update_value('%s%i' % (col_to_be_updated, i), None)
                    print("ERROR KeyError: %s" % e)


# Update stocks

What we are going to update:

In [5]:
r = YahooFinanceReader()


def update_price(ticket):
    """Apply the function for every ticker"""
    print("Update price for %s" % ticket)
    try:
      price = r.get_ticker(ticket).get_current_price()
      return price
    except KeyError as k:
      print(k)
      return None


cols_and_funcs = {
    COL_PRICE: update_price,
    COL_DIVS: lambda ticket: r.get_ticker(ticket).get_divs(),
    COL_REVENUE: lambda ticket: r.get_ticker(ticket).get_revenue(),
    COL_ASSETS: lambda ticket: r.get_ticker(ticket).get_assets(),
    COL_SHARES: lambda ticket: r.get_ticker(ticket).get_shares(),
    COL_NET: lambda ticket: r.get_ticker(ticket).get_net(),
    COL_AVG: lambda ticket: r.get_ticker(ticket).get_sma200days(),
    COL_PE: lambda ticket: r.get_ticker(ticket).get_pe(),
    COL_FPE: lambda ticket: r.get_ticker(ticket).get_fpe(),
    COL_FREECASH: lambda ticket: r.get_ticker(ticket).get_freecash(),
    COL_DEBT: lambda ticket: r.get_ticker(ticket).get_debt()
}


# Update excel

Updating gsheet for EU:

In [ ]:
sheet_ny = GSheetUpdater(NYSE_SHEET_NUMBER)
try:
  sheet_ny.apply_for_all(cols_and_funcs, 2)
except NoValue as e:
  print(e)

Index=2
Update price for RIO
Getting ticket object for RIO...
{'address1': '6 St James’s Square', 'city': 'London', 'zip': 'SW1Y 4AD', 'country': 'United Kingdom', 'phone': '44 20 7781 2000', 'website': 'https://www.riotinto.com', 'industry': 'Other Industrial Metals & Mining', 'industryKey': 'other-industrial-metals-mining', 'industryDisp': 'Other Industrial Metals & Mining', 'sector': 'Basic Materials', 'sectorKey': 'basic-materials', 'sectorDisp': 'Basic Materials', 'longBusinessSummary': 'Rio Tinto Group engages in exploring, mining, and processing mineral resources worldwide. The company operates through Iron Ore, Aluminium, Copper, and Minerals Segments. The Iron Ore segment engages in the iron ore mining, and salt and gypsum production in Western Australia. The Aluminum segment is involved in bauxite mining; alumina refining; and aluminium smelting. The Copper segment engages in mining and refining of copper, gold, silver, molybdenum, and other by-products and exploration activi

Getting ticket object for VZ...
{'address1': '1095 Avenue of the Americas', 'city': 'New York', 'state': 'NY', 'zip': '10036', 'country': 'United States', 'phone': '212 395 1000', 'website': 'https://www.verizon.com', 'industry': 'Telecom Services', 'industryKey': 'telecom-services', 'industryDisp': 'Telecom Services', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': 'Verizon Communications Inc., through its subsidiaries, engages in the provision of communications, technology, information, and entertainment products and services to consumers, businesses, and governmental entities worldwide. It operates in two segments, Verizon Consumer Group (Consumer) and Verizon Business Group (Business). The Consumer segment provides wireless services across the wireless networks in the United States under the Verizon and TracFone brands and through wholesale and other arrangements; and fixed wireless access (FW

Getting ticket object for VZ...
{'address1': '1095 Avenue of the Americas', 'city': 'New York', 'state': 'NY', 'zip': '10036', 'country': 'United States', 'phone': '212 395 1000', 'website': 'https://www.verizon.com', 'industry': 'Telecom Services', 'industryKey': 'telecom-services', 'industryDisp': 'Telecom Services', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': 'Verizon Communications Inc., through its subsidiaries, engages in the provision of communications, technology, information, and entertainment products and services to consumers, businesses, and governmental entities worldwide. It operates in two segments, Verizon Consumer Group (Consumer) and Verizon Business Group (Business). The Consumer segment provides wireless services across the wireless networks in the United States under the Verizon and TracFone brands and through wholesale and other arrangements; and fixed wireless access (FW

ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
Getting ticket object for USB...
{'address1': '800 Nicollet Mall', 'city': 'Minneapolis', 'state': 'MN', 'zip': '55402', 'country': 'United States', 'phone': '651-466-3000', 'website': 'https://www.usbank.com', 'industry': 'Banks - Regional', 'industryKey': 'banks-regional', 'industryDisp': 'Banks - Regional', 'sector': 'Financial Services', 'sectorKey': 'financial-services', 'sectorDisp': 'Financial Services', 'longBusinessSummary': 'U.S. Bancorp, a financial services holding company, provides various financial services to individuals, businesses, institutional organizations, governmental entities, and other financial institutions in the United States. It operates through Wealth, Corporate, Commercial and Institutional Banking; Consumer and Business Banking; Payment Services; and Treasury and Corporate Support segments. The company offers depository services, including checking accounts, savings accounts, and time certificate contracts; and 

ERROR KeyError: 'Net\xa0Tangible\xa0Assets'
Getting ticket object for USB...
{'address1': '800 Nicollet Mall', 'city': 'Minneapolis', 'state': 'MN', 'zip': '55402', 'country': 'United States', 'phone': '651-466-3000', 'website': 'https://www.usbank.com', 'industry': 'Banks - Regional', 'industryKey': 'banks-regional', 'industryDisp': 'Banks - Regional', 'sector': 'Financial Services', 'sectorKey': 'financial-services', 'sectorDisp': 'Financial Services', 'longBusinessSummary': 'U.S. Bancorp, a financial services holding company, provides various financial services to individuals, businesses, institutional organizations, governmental entities, and other financial institutions in the United States. It operates through Wealth, Corporate, Commercial and Institutional Banking; Consumer and Business Banking; Payment Services; and Treasury and Corporate Support segments. The company offers depository services, including checking accounts, savings accounts, and time certificate contracts; and 

Update price for CVU
Getting ticket object for CVU...
{'address1': '91 Heartland Boulevard', 'city': 'Edgewood', 'state': 'NY', 'zip': '11717', 'country': 'United States', 'phone': '631 586 5200', 'fax': '631 586 5840', 'website': 'https://www.cpiaero.com', 'industry': 'Aerospace & Defense', 'industryKey': 'aerospace-defense', 'industryDisp': 'Aerospace & Defense', 'sector': 'Industrials', 'sectorKey': 'industrials', 'sectorDisp': 'Industrials', 'longBusinessSummary': 'CPI Aerostructures, Inc. engages in the contract production of structural aircraft parts for fixed wing aircraft and helicopters in the commercial and defense markets. The company offers aerostructure and aerosystem assemblies, including new production and repair/overhaul of fielded wing structures and other control surfaces, rudder island, engine inlets/nacelles, engine exhaust manifolds, aircraft doors and windows, aircraft steps and racks, other aircraft secondary structures, airborne pod structures and integration of

In [ ]:
sheet_eu = GSheetUpdater(EU_SHEET_NUMBER)
try:
  sheet_eu.apply_for_all(cols_and_funcs, 2)
except NoValue as e:
  print(e)

Updating gsheet for NYSE:

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')